In [ ]:
## 5.4 卷积神经网络的可视化
### 5.4.1 可视化中间激活

In [ ]:
from keras.models import load_model

model = load_model('cats_and_dogs_small_2.h5')
model.summary()

**代码清单 5-25** 预处理单张图像

In [ ]:
img_path = '/Users/niujie/.keras/datasets/cats_and_dogs_train/cats_and_dogs_small/test/cats/cat.1700.jpg'

from keras.preprocessing import image
import numpy as np

img = image.load_img(img_path, target_size=(150, 150))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.  # 训练模型的输入数据都用这种方法预处理

# 形状为(1, 150, 150, 3)
print(img_tensor.shape)

**代码清单 5-26** 显示测试图像

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(img_tensor[0])
plt.show()

**代码清单 5-27** 用一个输入张量和一个输出张量列表将模型实例化

In [ ]:
from keras import models

layer_outputs = [layer.output for layer in model.layers[:8]]    # 提取前8层的输出
# 创建一个模型，给定模型输入，可以返回这些输出
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)

**代码清单 5-28** 以预测模式运行模型

In [ ]:
# 返回8个Numpy数组组成的列表，每个层激活对应一个Numpy数组
activations = activation_model.predict(img_tensor)

first_layer_activation = activations[0]
print(first_layer_activation.shape)

**代码清单 5-29** 将第4个通道可视化

In [ ]:
import matplotlib.pyplot as plt

plt.matshow(first_layer_activation[0, :, :, 4], cmap='viridis')

**代码清单 5-30** 将第7个通道可视化

In [ ]:
plt.matshow(first_layer_activation[0, :, :, 7], cmap='viridis')

**代码清单 5-31** 将每个中间激活的所有通道可视化

In [ ]:
# 层的名称，可以将这些名称画到图中
layer_names = []
for layer in model.layers[:8]:
    layer_names.append(layer.name)

images_per_row = 16

# 显示特征图
for layer_name, layer_activation in zip(layer_names, activations):
    n_features = layer_activation.shape[-1]     # 特征图中的特征个数

    size = layer_activation.shape[1]            # 特征图的形状为(1, size, size, n_features)

    n_cols = n_features // images_per_row       # 在这个矩阵中将激活通道平铺
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    # 将每个过滤器平铺到一个大的水平网格中
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0, :, :, col * images_per_row + row]
            # 对特征进行后处理，使其看起来更美观
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image
    
    # 显示网格
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

plt.show()